# CSCI 6390: Assignment 5
## Due October 24th
### By: Nicholas Lutrzykowski 
The goal of this assignment is to use SVM 


In [57]:
# Import Statements 
import numpy as np
import csv
import matplotlib.pyplot as plt
import warnings 
from sklearn.preprocessing import StandardScaler

### Import and setup data 

In [58]:
data_orig = np.genfromtxt('OnlineNewsPopularity.csv', delimiter=',')

data = np.concatenate((data_orig[1:,2:4], data_orig[1:, 7:13], data_orig[1:, 39:61]), axis=1)

num_points = data.shape[0]
num_attributes = data.shape[1]
print("Number of attributes:", num_attributes)
print("Number of points:", num_points)

# Read in the titles 
file = open('OnlineNewsPopularity.csv')
csvreader = csv.reader(file)
header = [] 
header = next(csvreader)
header = header[2:13] + header[39:]

Number of attributes: 30
Number of points: 39644


### Data Preprocessing

In [59]:
def get_target(target):
    target = np.where(target < 6000, target, 6000)
    target = np.where(target < 2000, -1, target)
    target = np.where(target > 2000, 1, target)
    
    
    return target 

In [60]:
np.random.seed(42)
np.random.shuffle(data)
np.random.shuffle(data)
num_training, num_validation, num_test = 1000, 2000, 3000


training = data[:num_training, :]
validation = data[num_training:num_validation, :]
test = data[num_validation:num_test, :]
training_target, validation_target, test_target = training[:, -1], validation[:, -1], test[:, -1]
training = training[:, :-1]
validation = validation[:, :-1]
test = test[:, :-1]

training_target, validation_target, test_target = get_target(training_target), get_target(validation_target), get_target(test_target)

scaler = StandardScaler(with_mean=True, with_std=True)
scaler.fit(training)
training = scaler.transform(training)
validation = scaler.transform(validation)
test = scaler.transform(test)

print("Training shape:", training.shape)
print("Validation shape:", validation.shape)
print("Test shape:", test.shape)


Training shape: (1000, 29)
Validation shape: (1000, 29)
Test shape: (1000, 29)


### Part I: SVM

In [61]:
def SVM(data, y, K, C, epsilon):
    K += 1
    eta = 1/K.diagonal()
    
    t = 0
    alpha = np.zeros((data.shape[0],))
    alpha_prev = alpha
    
    dif = 2*epsilon
    while dif > epsilon: 
        for k in range(data.shape[0]):
            temp = np.multiply(np.multiply(alpha, y), K[:,0])
            alpha[k] = alpha[k] + eta[k]*(1 - y[k]*np.sum(temp))
            
            if alpha[k] < 0: alpha[k] = 0 
            if alpha[k] > C: alpha[k] = C
            
        dif = np.linalg.norm(alpha-alpha_prev)
        alpha_prev = alpha
    
    w = np.matmul(np.multiply(alpha, y), data)
    return w 

In [62]:
def find_accuracy(data, y, w): 
    w = np.reshape(w, (w.shape[0], 1))
    y_hat = np.matmul(data, w)
    y_hat = np.where(y_hat < 0, -1, 1)
    y_hat = np.reshape(y_hat, (data.shape[0],))
    accuracy = 1 - (data.shape[0] - np.count_nonzero(y_hat-y))/data.shape[0]
    
    return accuracy
    

#### Linear Kernal 

In [63]:
C = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]
epsilon = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]

linear_K = np.matmul(training, training.T) + 1 

opt_accuracy, opt_C, opt_eps, opt_w = 0, 0, 0, np.zeros((training.shape[1],))

for c_val in C: 
    for eps_val in epsilon: 
        w = SVM(training, training_target, linear_K, c_val, eps_val)

        accuracy = find_accuracy(validation, validation_target, w)
        
        if accuracy > opt_accuracy: 
            opt_accuracy, opt_C, opt_eps, opt_w = accuracy, c_val, eps_val, w

print("Linear kernal results:")
print("Accuracy:", round(find_accuracy(test, test_target, opt_w), 3))
print("Optimal C is:", opt_C)
print("Optimal epsilon is:", opt_eps)


Linear kernal results:
Accuracy: 0.529
Optimal C is: 0.001
Optimal epsilon is: 0.1


#### Gaussian Kernal

In [64]:
def reduce_dim(eigvalues, eigvectors, K_bar):
    alpha = 0.95 
    f_r = 0 
    r = 1
    while f_r < alpha: 
        f_r = np.sum(eigvalues[:r])/np.sum(eigvalues)
        r += 1

    C_r = eigvectors[:, :r]
    A = np.matmul(K_bar, C_r)
    
    return f_r, r, C_r, A

In [65]:
def center_kernal(K):
    # Center the Kernal Matrix 
    num_points = K.shape[0]
    I = np.identity(num_points)
    ones = np.full((num_points, num_points), 1/num_points, dtype=float)

    # Check that k_bar is found correctly 
    K_bar = np.matmul(np.matmul(I-ones, K), I-ones)

    eigvalues, eigvectors = np.linalg.eigh(K_bar)
    # Reverse the eigenvalues and eigenvectors to be in increasing order

    eigvalues = np.flip(eigvalues, axis=0)
    eigvectors = np.flip(eigvectors, axis=1)

    variances = eigvalues/num_points 
    '''Some of the eigvalues are currently negative, we need to figure out what is going wrong'''
    # Before finding ci ensure that ui = 1
    eigvalues = np.where(eigvalues > 0, eigvalues, 1E-9)
    #print(np.sqrt(1/eigvalues))
    ci = np.sqrt(1/eigvalues)*eigvectors
    
    f_r, r, C_r, A = reduce_dim(eigvalues, ci, K_bar)
    
    return f_r, r, C_r, A
    

In [66]:
mu = np.sum(training, axis=0)/num_points
S = np.sqrt(np.sum(training**2, axis=1))
sigma = 400000
f_r_max = 0
sigma_optimal = 0
# 40000 to 40010 
# Loop over values of sigma to find the optimal sigma value
while sigma < 400010:  
    gaussian_kernal = np.exp((2*np.dot(training, training.T)-S-S.T)/(2*sigma))

    f_r, r, C_r, A = center_kernal(gaussian_kernal)
    
    if f_r > f_r_max: 
        sigma_optimal = sigma
        f_r_max = f_r
    
    sigma += 2

print("The optimal sigma is:")
print(sigma_optimal)

gaussian_kernal = np.exp((2*np.dot(training, training.T)-S-S.T)/(2*sigma_optimal))

The optimal sigma is:
400008


In [67]:
C = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]
epsilon = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]

opt_accuracy, opt_C, opt_eps, opt_w = 0, 0, 0, np.zeros((training.shape[1],))

for c_val in C: 
    for eps_val in epsilon: 
        w = SVM(training, training_target, gaussian_kernal, c_val, eps_val)

        accuracy = find_accuracy(validation, validation_target, w)
        
        if accuracy > opt_accuracy: 
            opt_accuracy, opt_C, opt_eps, opt_w = accuracy, c_val, eps_val, w

print("Gaussian kernal results:")
print("Accuracy:", round(find_accuracy(test, test_target, opt_w), 3))
print("Optimal C is:", opt_C)
print("Optimal epsilon is:", opt_eps)

Gaussian kernal results:
Accuracy: 0.483
Optimal C is: 1000
Optimal epsilon is: 1


#### CSCI 6390
Implement the inhomogeneous polynomial kernel. You need to select the best degree and CC value.

In [68]:
C = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]
epsilon = [1e-3, 1e-2, 0.1, 1, 10, 100, 1000]

opt_accuracy, opt_C, opt_eps, opt_w, opt_degree, opt_constant = 0, 0, 0, np.zeros((training.shape[1],)), 0, 0
# C is a hyperparameter, so we must loop to find the optimal value 

C_max = 380
C_kernal = 290


degrees = [1, 2, 3, 4, 5, 6]
for i in degrees: 
    while C_kernal < C_max:
        poly_kernal = (C_kernal+np.matmul(training, training.T))**i
        
        for c_val in C: 
            for eps_val in epsilon: 
                w = SVM(training, training_target, poly_kernal, c_val, eps_val)

                accuracy = find_accuracy(validation, validation_target, w)
                
                if accuracy > opt_accuracy: 
                    opt_accuracy, opt_C, opt_eps, opt_w, opt_degree, opt_constant = accuracy, c_val, eps_val, w, i, C_kernal
                
        C_kernal += 10

optimal_kernal = (opt_constant+np.matmul(training, training.T))**opt_degree

print("Inhomogeneous Polynomial kernal results:")
print("Accuracy:", round(find_accuracy(test, test_target, opt_w), 3))
print("Optimal C is:", opt_C)
print("Optimal epsilon is:", opt_eps)
print("Optimal Constant C is:", opt_constant)
print("Optimal degree is:", opt_degree)


Inhomogeneous Polynomial kernal results:
Accuracy: 0.533
Optimal C is: 1000
Optimal epsilon is: 0.1
Optimal Constant C is: 300
Optimal degree is: 1


### Part II: Exam I Questions

In [25]:
data = np.loadtxt('./seeds_dataset.txt')
num_points = data.shape[0]
num_attributes = data.shape[1]

print("The shape of the data matrix is: ", data.shape)
print("The number of attributes is:", num_attributes)
print("The number of points is:",num_points)

The shape of the data matrix is:  (210, 8)
The number of attributes is: 8
The number of points is: 210


#### Q5 (20 points) Non-linear direction in input space: For the polynomial kernel it is actually possible to compute the direction of the kernel principal component. For the homogeneous quadratic kernel, compute the best quadratic PC direction equation.

In [29]:
# We know any direction is a linear combination of points 
quadratic_kernal = (np.matmul(data, data.T))**2
f_r, r, C_r, A = center_kernal(quadratic_kernal)

np.random.seed(42)
eps = 1e-4

# Initialize X0
Xprev = np.random.rand(num_points, 2) 
Xprev[:,0], Xprev[:,1] = Xprev[:,0]/np.linalg.norm(Xprev[:,0]), Xprev[:,1]/np.linalg.norm(Xprev[:,1])

eiga, eigb = 0, 0 

# Find the convergence 
dif = 2*eps
while(dif > eps): 
    Xi = np.matmul(quadratic_kernal, Xprev)

    # Orthogonalize the columns 
    a, b = np.reshape(Xi[:,0], (num_points, 1)), np.reshape(Xi[:,1], (num_points, 1))
    b = b - (np.matmul(b.T, a)/np.dot(a.T, a)*a)

    # Normalize the columns 
    a, b = a/np.linalg.norm(a), b/np.linalg.norm(b)
    Xi[:, 0], Xi[:, 1]= np.reshape(a, (num_points,)), np.reshape(b, (num_points,))

    dif = np.linalg.norm(Xprev-Xi)
    Xprev = Xi 

eigvectors_new = Xprev

# Find the eigenvalues (there is no need to divide by length of eigenvector since it is normalized)
eiga = np.dot(np.matmul(quadratic_kernal, eigvectors_new[:, 0]), eigvectors_new[:, 0])  
eigb = np.dot(np.matmul(quadratic_kernal, eigvectors_new[:, 1]), eigvectors_new[:, 1])  

print("Direction of Kernal Principal Component:")
print(eigvectors_new[:, 0])

Direction of Kernal Principal Component:
[0.06648823 0.06243041 0.05948238 0.05703907 0.06970657 0.06029944
 0.06392164 0.05922553 0.07555933 0.07321256 0.0685746  0.05827729
 0.05882851 0.05805518 0.05762364 0.06251495 0.05935507 0.06688338
 0.06038418 0.05312137 0.06115463 0.06006921 0.06761577 0.04767448
 0.06455941 0.07044301 0.0542909  0.05235173 0.05968839 0.05731687
 0.05330617 0.06885045 0.06192749 0.05848585 0.06515699 0.07086492
 0.07297094 0.07726552 0.06430687 0.06342393 0.05626048 0.05584997
 0.05313454 0.07036498 0.06543768 0.05675541 0.06570347 0.06481492
 0.06350824 0.06428204 0.0628991  0.07166263 0.06436914 0.06149572
 0.06235006 0.06522933 0.06152957 0.06220119 0.06645448 0.04820491
 0.04481283 0.04323787 0.04946429 0.05624041 0.05093028 0.05166097
 0.0602224  0.05941395 0.06060974 0.05358426 0.08489106 0.08018873
 0.08202724 0.09180729 0.07880803 0.07965939 0.08246567 0.10644735
 0.09487004 0.07881178 0.07820644 0.09157004 0.10232914 0.09498377
 0.09599312 0.0867897

#### Q8 (**CSCI 6390 Only**: 10 points) Let $\mathbf{a}$ be random $d$-dimensional binary vector, and $\mathbf{b}$ a random $d$-dimensional corner in a $d$-dimensional unit hypercube (with the range in each axis as $[0,1]$). Derive the formula for the expected angle between $\mathbf{a}$ and $\mathbf{b}$? 

We know that a is a d-dimensional binary vector and lets assume that b is a random d-dimensional coordinates of a corner in a d-dimensional hyper cube. By placing the origin at the center of the cube, as well as the a vector start point we find that the resulting expected angle between a and b will be $\theta = cos^{-1}\frac{ab}{|a||b|}$. 